# Kociemba's Two-Phase Algorithm - Detailed Analysis

**Author:** Alex Toska  
**Affiliation:** University of Patras  
**Algorithm:** Kociemba (1992)  

---

## Overview

Herbert Kociemba's two-phase algorithm is an improvement over Thistlethwaite, reducing from 4 phases to 2 while achieving near-optimal solutions (≤19 moves).

This notebook covers:
- Two-phase structure and theory
- Coordinate systems (6 coordinates)
- Pruning tables and optimizations
- Near-optimal solution finding
- Comparison with Thistlethwaite and Korf

## Setup

In [ ]:
import sys
from pathlib import Path
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src.cube.rubik_cube import RubikCube
from src.kociemba.solver import KociembaSolver
from src.cube.visualization import display_cube_unfolded
from src.cube.visualize_3d import visualize_cube_3d

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 5)

print("✓ Imports successful!")

## 1. Two-Phase Algorithm Theory

Kociemba simplified Thistlethwaite's 4 phases into 2 phases, achieving better solutions.

### The Two Phases

**Phase 1: G₀ → G₁** (Build subgroup G₁)  
**Goal:** Orient all pieces and position middle layer edges

- All edge orientations correct
- Middle layer edges (FR, FL, BR, BL) in E-slice
- All corner orientations correct

**Allowed moves:** All 18 moves  
`U, U2, U', D, D2, D', F, F2, F', B, B2, B', L, L2, L', R, R2, R'`

**Phase 2: G₁ → G₂** (Solve within G₁)  
**Goal:** Solve the cube using only G₁ moves

- Maintain orientation and E-slice constraints
- Permute all pieces to solved state

**Allowed moves:** 10 moves  
`U, U2, U', D, D2, D', F2, B2, L2, R2`

### Key Difference from Thistlethwaite

| Feature | Thistlethwaite | Kociemba |
|---------|----------------|----------|
| Phases | 4 | 2 |
| G₁ Definition | Edges oriented | Edges oriented + corners oriented + E-slice |
| Search Strategy | BFS per phase | IDA* with pruning |
| Solution | 30-52 moves | ≤19 moves |
| Time | 0.1-2s | 0.001-5s |

## 2. Coordinate Systems

Kociemba uses 6 coordinates to represent cube state:

### Phase 1 Coordinates (3 total)

1. **Edge Orientation (EO):** 2¹¹ = 2,048 states
   - Orientation of 11 edges (12th is derived)
   - Binary: good or bad orientation

2. **Corner Orientation (CO):** 3⁷ = 2,187 states
   - Orientation of 7 corners (8th is derived)
   - Ternary: 3 possible orientations per corner

3. **E-Slice Position:** C(12,4) = 495 states
   - Which 4 edges are in middle layer
   - Combination, not permutation

**Phase 1 state space:** 2,048 × 2,187 × 495 ≈ **2.2 billion** states

### Phase 2 Coordinates (3 total)

4. **Corner Permutation (CP):** 8! = 40,320 states
   - Permutation of all 8 corners

5. **UD Edge Permutation:** 8! = 40,320 states
   - Permutation of 8 edges in U and D layers

6. **E-Slice Permutation:** 4! = 24 states
   - Permutation of 4 middle layer edges

**Phase 2 state space:** 40,320 × 40,320 × 24 ≈ **39 billion** states (but constrained by G₁)

## 3. Pruning Tables

Kociemba uses pruning tables to accelerate IDA* search:

### Phase 1 Pruning Tables

1. **EO + CO pruning:** 2,048 × 2,187 = 4,478,976 entries
   - Pre-computed minimum distance for each (EO, CO) pair
   - Size: ~4 MB

2. **CO + E-Slice pruning:** 2,187 × 495 = 1,082,565 entries
   - Minimum distance for each (CO, E-Slice) pair
   - Size: ~1 MB

3. **EO + E-Slice pruning:** 2,048 × 495 = 1,013,760 entries
   - Minimum distance for each (EO, E-Slice) pair
   - Size: ~1 MB

**Phase 1 tables:** ~6 MB total

### Phase 2 Pruning Tables

1. **CP + UD Edge pruning:** Reduced table (~20 MB)
2. **CP + E-Slice pruning:** Small table (~1 MB)
3. **UD Edge + E-Slice pruning:** Small table (~1 MB)

**Phase 2 tables:** ~22 MB total

**Total memory:** ~80 MB (including move tables)

## 4. Solving Example

In [ ]:
# Create and scramble a cube
cube = RubikCube()
scramble = cube.scramble(moves=15, seed=42)

print("=== SCRAMBLE ===")
print(f"Scramble: {' '.join(scramble)}")
print(f"Scramble length: {len(scramble)} moves\n")

# Visualize
fig = visualize_cube_3d(cube, view_angles=(30, 45))
fig.suptitle("Scrambled Cube (15 moves)", fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# Solve using Kociemba
solver = KociembaSolver(max_depth_phase1=12, max_depth_phase2=18)

print("=== SOLVING WITH KOCIEMBA ===")
start_time = time.time()
solution = solver.solve(cube)
end_time = time.time()

print(f"\n✓ Solution found!")
print(f"Solution: {' '.join(solution)}")
print(f"Solution length: {len(solution)} moves")
print(f"Time: {end_time - start_time:.4f} seconds")

# Verify
test_cube = cube.copy()
for move in solution:
    test_cube.apply_move(move)

print(f"\nVerification: Cube is {'SOLVED ✓' if test_cube.is_solved() else 'NOT SOLVED ✗'}")

In [ ]:
# Visualize solved cube
fig = visualize_cube_3d(test_cube, view_angles=(30, 45))
fig.suptitle("Solved Cube", fontsize=14, fontweight='bold')
plt.show()

## 5. Phase Distribution

Unlike Thistlethwaite's 4 phases, Kociemba has just 2:

### Typical Distribution

- **Phase 1:** 7-11 moves (reach G₁)
- **Phase 2:** 7-12 moves (solve in G₁)
- **Total:** 14-19 moves

### Solution Quality

- **Best case:** ~11 moves
- **Average:** 16-18 moves
- **Worst case:** 19 moves (proven upper bound)
- **Optimal (God's Number):** 20 moves

Kociemba typically produces solutions within 1-2 moves of optimal!

## 6. Performance Benchmark

In [ ]:
# Benchmark on 10 scrambles
print("Running benchmark on 10 scrambles...\n")

results = []
for seed in range(42, 52):
    cube = RubikCube()
    scramble = cube.scramble(moves=20, seed=seed)
    
    solver = KociembaSolver(max_depth_phase1=12, max_depth_phase2=18)
    start = time.time()
    solution = solver.solve(cube)
    elapsed = time.time() - start
    
    results.append({
        'seed': seed,
        'scramble_length': len(scramble),
        'solution_length': len(solution),
        'time_seconds': elapsed
    })
    
    print(f"Seed {seed}: {len(solution)} moves in {elapsed:.3f}s")

# Create DataFrame
df = pd.DataFrame(results)

print("\n=== Summary Statistics ===")
print(f"Average solution length: {df['solution_length'].mean():.1f} moves")
print(f"Min solution length: {df['solution_length'].min()} moves")
print(f"Max solution length: {df['solution_length'].max()} moves")
print(f"Average time: {df['time_seconds'].mean():.4f} seconds")
print(f"Min time: {df['time_seconds'].min():.4f} seconds")
print(f"Max time: {df['time_seconds'].max():.4f} seconds")

In [ ]:
# Visualize results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Solution lengths
ax1.bar(df['seed'] - 42, df['solution_length'], color='#2ecc71', alpha=0.7)
ax1.axhline(y=df['solution_length'].mean(), color='red', linestyle='--', 
            label=f"Mean: {df['solution_length'].mean():.1f}")
ax1.axhline(y=20, color='orange', linestyle=':', 
            label="God's Number: 20")
ax1.set_xlabel('Test Case', fontsize=12)
ax1.set_ylabel('Solution Length (moves)', fontsize=12)
ax1.set_title('Kociemba Solution Lengths', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Execution times
ax2.bar(df['seed'] - 42, df['time_seconds'], color='#3498db', alpha=0.7)
ax2.axhline(y=df['time_seconds'].mean(), color='red', linestyle='--',
            label=f"Mean: {df['time_seconds'].mean():.3f}s")
ax2.set_xlabel('Test Case', fontsize=12)
ax2.set_ylabel('Time (seconds)', fontsize=12)
ax2.set_title('Kociemba Execution Time', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Comparison: Thistlethwaite vs Kociemba

In [ ]:
# Quick comparison on same scramble
from src.thistlethwaite.solver import ThistlethwaiteSolver

cube = RubikCube()
cube.scramble(moves=15, seed=100)

# Thistlethwaite
t_solver = ThistlethwaiteSolver(use_pattern_databases=False)
t_start = time.time()
t_solution = t_solver.solve(cube.copy())
t_time = time.time() - t_start

# Kociemba
k_solver = KociembaSolver(max_depth_phase1=12, max_depth_phase2=18)
k_start = time.time()
k_solution = k_solver.solve(cube.copy())
k_time = time.time() - k_start

# Display comparison
comparison_data = [
    {
        "Algorithm": "Thistlethwaite",
        "Solution Length": len(t_solution),
        "Time (s)": f"{t_time:.4f}",
        "Phases": 4
    },
    {
        "Algorithm": "Kociemba",
        "Solution Length": len(k_solution),
        "Time (s)": f"{k_time:.4f}",
        "Phases": 2
    }
]

comp_df = pd.DataFrame(comparison_data)
print("\n=== Thistlethwaite vs Kociemba ===")
print(comp_df.to_string(index=False))

print(f"\nKociemba uses {len(t_solution) - len(k_solution)} fewer moves")
print(f"Solution quality improvement: {(1 - len(k_solution)/len(t_solution)) * 100:.1f}%")

In [ ]:
# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

algorithms = ['Thistlethwaite', 'Kociemba']
moves = [len(t_solution), len(k_solution)]
times = [t_time, k_time]
colors = ['#3498db', '#2ecc71']

# Solution length
bars1 = ax1.bar(algorithms, moves, color=colors, alpha=0.7)
ax1.set_ylabel('Moves', fontsize=12)
ax1.set_title('Solution Length Comparison', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}', ha='center', va='bottom', fontsize=12)

# Time
bars2 = ax2.bar(algorithms, times, color=colors, alpha=0.7)
ax2.set_ylabel('Seconds', fontsize=12)
ax2.set_title('Execution Time Comparison', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}s', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

## 8. Strengths & Weaknesses

### ✅ Strengths

1. **Near-optimal solutions:** ≤19 moves (vs 20 optimal)
2. **Better than Thistlethwaite:** ~40% fewer moves
3. **Reasonable speed:** 0.001-5 seconds typical
4. **Practical for competition:** Used in actual speed cubing software
5. **Good balance:** Speed vs quality trade-off
6. **Proven upper bound:** Mathematical guarantee of ≤19 moves

### ❌ Weaknesses

1. **Memory intensive:** ~80 MB (pruning tables)
2. **Complex implementation:** 6 coordinate systems
3. **Not always optimal:** Can be 1-2 moves longer than God's Number
4. **Variable speed:** Some positions take longer
5. **Setup overhead:** Loading pruning tables takes time

### 🎯 Best Use Cases

- **Speed cubing software** - Used in real solvers
- **Practical solving** - Good enough for most purposes
- **Competition benchmarks** - Industry standard
- **When optimality matters** - But not at any cost
- **Modern applications** - Current state-of-the-art for practical use

## 9. Algorithm Comparison Table

### Complete Comparison: Thistlethwaite vs Kociemba vs Korf

| Feature | Thistlethwaite | Kociemba | Korf |
|---------|----------------|----------|------|
| **Year** | 1981 | 1992 | 1997 |
| **Phases** | 4 | 2 | 1 (IDA*) |
| **Solution Length** | 30-52 moves | ≤19 moves | ≤20 moves |
| **Typical Length** | 35-45 | 16-18 | 18-20 |
| **Speed** | ⚡ Very Fast (0.1-2s) | 🚀 Medium (0.001-5s) | 🐢 Slower (1-30s) |
| **Memory** | 💾 Low (~2 MB) | 💾 High (~80 MB) | 💾 Medium (~45 MB) |
| **Optimality** | ❌ Sub-optimal | ⚠️ Near-optimal | ✅ Optimal |
| **Phases** | G₀→G₁→G₂→G₃→G₄ | G₀→G₁→G₂ | Single search |
| **Search** | BFS per phase | IDA* with pruning | IDA* with pruning |
| **Pruning** | Minimal | Extensive (~80 MB) | Pattern DB (~45 MB) |
| **Complexity** | Medium | High | Very High |
| **Use Case** | Education, demos | Practical solving | Research, optimal |
| **Competition** | No | Yes | No (too slow) |

## 10. Implementation Details

### Project Structure

The implementation includes:

1. **Coordinates** (`src/kociemba/coordinates.py`)
   - Edge orientation, corner orientation, E-slice
   - Corner permutation, UD edge permutation, E-slice permutation

2. **Move Tables** (`src/kociemba/move_tables.py`)
   - Pre-computed coordinate transitions
   - Separate tables for Phase 1 and Phase 2

3. **Pruning Tables** (`src/kociemba/pruning_tables.py`)
   - Phase 1: EO+CO, CO+E-slice, EO+E-slice
   - Phase 2: CP+UD, CP+E-slice, UD+E-slice

4. **Solver** (`src/kociemba/solver.py`)
   - Two-phase IDA* search
   - Depth management and pruning

### Usage Example

```python
from src.cube.rubik_cube import RubikCube
from src.kociemba.solver import KociembaSolver

# Create and scramble
cube = RubikCube()
cube.scramble(moves=20, seed=42)

# Solve with custom depths
solver = KociembaSolver(
    max_depth_phase1=12,  # Max depth for phase 1
    max_depth_phase2=18   # Max depth for phase 2
)
solution = solver.solve(cube)

print(f"Solution: {' '.join(solution)}")
print(f"Length: {len(solution)} moves")
```

## 11. Exercises

### Exercise 1: Depth Limits
Test how depth limits affect solution quality and speed:
```python
for d1 in [10, 11, 12, 13]:
    for d2 in [16, 17, 18]:
        solver = KociembaSolver(max_depth_phase1=d1, max_depth_phase2=d2)
        # Compare results...
```

### Exercise 2: Hard Scrambles
Find scrambles that take longest to solve:
```python
max_time = 0
worst_seed = None
for seed in range(1000):
    cube = RubikCube()
    cube.scramble(moves=20, seed=seed)
    start = time.time()
    solution = solver.solve(cube)
    elapsed = time.time() - start
    if elapsed > max_time:
        max_time = elapsed
        worst_seed = seed
```

### Exercise 3: Phase Analysis
Modify solver to report phase 1 vs phase 2 move counts:
```python
# Track phase boundaries
# Analyze typical distribution
# Find cases with long phase 1 or phase 2
```

### Exercise 4: Memory Profiling
Profile memory usage of pruning tables:
```python
import sys
# Measure size of each pruning table
# Compare with Thistlethwaite
```

## 12. References

### Original Papers

1. **Kociemba, H. (1992)** - "Close to God's Algorithm"
   - Original two-phase algorithm description
   - Proof of ≤19 move upper bound

2. **Korf, R. E. (1997)** - "Finding Optimal Solutions to Rubik's Cube"
   - Comparison of Kociemba with optimal solving
   - Pattern database improvements

3. **Rokicki, T., et al. (2014)** - "The Diameter of the Rubik's Cube Group is Twenty"
   - Proof that God's Number is 20
   - Shows Kociemba is within 1 move of optimal bound

### Implementation Resources

- Project source: `src/kociemba/`
- Tests: `tests/kociemba/`
- Benchmarks: `results/phase8_comprehensive_benchmark.json`
- Online solver: http://kociemba.org/cube.htm

### Additional Reading

- Coordinate systems: https://www.jaapsch.net/puzzles/compcube.htm
- Implementation guide: https://github.com/hkociemba/RubiksCube-TwophaseSolver
- Comparison notebook: `05_Algorithm_Comparison.ipynb`

## Conclusion

Kociemba's algorithm represents a major advancement in practical Rubik's Cube solving.

### Key Takeaways

1. **Near-optimal is often enough** - 16-18 moves vs 20 optimal
2. **Two phases better than four** - Simplification improves results
3. **Pruning is crucial** - 80 MB of tables accelerates search
4. **Industry standard** - Used in real speed cubing software
5. **Sweet spot** - Best balance of speed and solution quality

### Why Kociemba Dominates

- **Practical:** Fast enough for real-time use
- **Effective:** Solutions close to optimal
- **Proven:** Mathematical guarantees (≤19 moves)
- **Scalable:** Can trade depth for speed

### Next Steps

- Explore **04_Korf.ipynb** - See how to achieve true optimality
- Explore **05_Algorithm_Comparison.ipynb** - Full benchmark comparison
- Explore **06_Conclusion.ipynb** - Overall project summary

---

**Happy Cubing! 🎲**